In [ ]:
addprocs()
using SortSpikes, ExtractSpikes, DistributedArrays

In [ ]:
#Parallel Processing template

st=Array(Sorting,1024)
for i=1:1024
    st[i]=Sorting(SpikeDetection(),Cluster(),ones(Int64,20000),zeros(Int,500),zeros(Int,500),2,zeros(Int,50,100))
    st[i].rawSignal[200:700:end]=10*st[i].rawSignal[200:700:end]
end

@time s=distribute(st);

@time map!(onlineCal, s);
@time map!(onlineSort, s);

In [ ]:
hi=rand(Int,20000,1024)
hi=convert(SharedArray{Int,2},hi)

@time @parallel for i=1:1024
    for j=1:20000
        s[i].rawSignal[j]=hi[j,i]
    end
end

In [ ]:
#pyqtgraph testing
using PySide
reload(Pkg.dir("PySide", "src", "pyqtgraph.jl"))
using PyQtGraph
using PyCall

In [ ]:
w = Widget()
lyt = GridLayout(w)
#setLayout(w)

set_size(w, 800, 600)

#shows window
raise(w)

# Graphics Layout Widget make grid of plot devices
win = GraphicsLayoutWidget(w)
addWidget(lyt, win)

# addPlot adds a plot device in next column over and returns a PyObject
# PyQtGraph.GraphicsPlot object
p = [win[:addPlot](i,j) for i=1:4, j=1:8]

#stores all of the curves that can be present on the plot
#k dictates how many curves can be present at once
curve = [p[i,j][:plot](pen="y") for i=1:4, j=1:8, k=1:5]

data = rand(100,10);
ptr = 0;
h=ones(Int,4,8);

In [ ]:
function update()
    global curve, data, ptr, p
    for i=1:4,j=1:8
        qinvoke(curve[i,j,h[i,j]], :setData, data[:, 1 + ptr])
        p[i,j][:enableAutoRange]("xy", false) ## stop auto-scaling
        ptr = (ptr + 1) % 10
        h[i,j]+=1
        if h[i,j]>5
            h[i,j]=1
            for k=1:5
                qinvoke(curve[i,j,k], :clear)
            end
        end       
    end
end

#creates a timer
timer = QtCore[:QTimer]()

#connects the action that will occur when the timer triggers
qconnect(timer, :timeout, update)

#sets the interval between timer actions in ms; 0 for continuous 
#(as fast as can be processed)
qinvoke(timer, :start, 200)

In [ ]:
#GUI Experiments

# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

In [ ]:
#Wavelet Experiments

function cwt(x::Array{Float64,1},t::Int,psi::Array{Float64,1},bs::Array{Float64,1})

    bs[1]=bs[2]-x[t-20]*psi[1]+x[t]*psi[21]

    for i=2:19
        bs[i]=bs[i+1]-x[t-20]*psi[i]+x[t]*psi[22-i] 
    end

    bs[20]=0.0
    for i=1:20
        bs[20]+=x[t-20+i]*psi[21-i]
    end
    
    return bs
end


function mycwt(x::Array{Float64,1},t::Int, psi::Array{Float64,1})
    
    bs=0.0
    
    for i=1:20
        bs+=x[t-20+i]*psi[21-i]
    end
    
end

function main()
    for i=1:20000
        mycwt(x,t,psi)
    end
end

psi=[0.00021208083980379827
0.00035858968789573785
-0.0021782363581090178
-0.004159358781386048
0.010131117519849788
0.023408156785839195
-0.02816802897093635
-0.09192001055969624
0.05204316317624377
0.4215662066908515
-0.7742896036529562
0.4379916261718371
0.06203596396290357
-0.10557420870333893
-0.0412892087501817
0.03268357426711183
0.01976177894257264
-0.009164231162481846
-0.006764185448053083
0.0024333732126576722
0.0016628637020130838
-0.0006381313430451114
-0.00030225958181306315
0.00014054114970203437
4.134043227251251e-05
-2.1315026809955787e-05
-3.7346551751414047e-06
2.0637618513646814e-06
1.6744288576823017e-07
-9.517657273819165e-08
];

bs=zeros(Float64,20)
    
x=rand(100)
t=50

for j=1:20
    for i=1:20
        bs[j]+=x[t-20+i]*psi[21-i]
    end
end